# NASA International Space Apps Challenge - InSAR Change Detectives!

Team LTI Software and Engineering:

 - Alexandre Beaulieu
 - Mathieu Bergeron
 - Gabriel Gibeau Sanchez
 - Olivier Lavergne
 - Maxime Rondeau

## Overview
Here are the processing steps.

### Data source
#### Satellite products
We use L1 data from the SENTINEL-1A Interferometric Wide Swath Level 1 S Product. Data are downloaded via the [python scripts](data/download_2015NepalEarthquake.py) provided by the Alaska Satellite Facility's [bulk download service][asf].

#### Weather data
We use weather data from the [Copernicus Climate Change Service (C3S) Climate Data Store][atmopspheric-data]. There is a utility scrip to download the atmospheric data corresponding to three geological events of interest; namely the Ridgecrest, CA, USA earthquakes of July 4th and 5th 2019, the Kilauea, HI, USA volcanic eruption of May 4th 2018 and the Kumamoto, KYU, Japan earthquake of April 16th 2016. The files have also been pushed to the repository for your convinience.

##### ERA5 data download script

Once the steps (found [here](https://confluence.ecmwf.int/display/CKB/How+to+install+and+use+CDS+API+on+Windows) *Note: The .cdspapirc file need to be created manually for Windows user) to create a login for ERA5's API have been completed, atmospheric data can be easily downloaded the query(ies) string(s) in the `ERA5APICalls.py` script. Example of queries can be obtained from https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=form.

### Processing steps
To generate an interferogram, Level 1 data is preprocessed using the following steps, similar the example found at [this repository][sentinel-1-snappy].
 1.  Open source files
 2.  Select chosen subswath
 3.  Apply Geocoding
 4.  Burst co-registration using orbit and (digital elevation map) DEM
 5.  Estimate constant range and azimuth offsets for the whole image
 6.  Compute interferogram
 7.  Debursts the TOPSAR product
 8.  Compute and subtract topographic phase
 9.  Multilook: Average spectra over a set number of bands
 10. Apply terrain corrections to remove shadows and order terrain-induces artefacts


[asf]:               https://asf.alaska.edu/how-to/data-tools/data-tools/#bulk_download
[sentinel-1-snappy]: https://github.com/crisjosil/InSAR_Snappy
[atmospheric-corrections]: https://doi.org/10.1016/j.jappgeo.2009.03.010
[atmopspheric-data]: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview "Muñoz Sabater, J., (2021) was downloaded from the Copernicus Climate Change Service (C3S) Climate Data Store"


## Prerequisites
The following software are required on your platform:
 - A [conda](https://www.anaconda.com/products/distribution) (or [mamba](https://github.com/mamba-org/mamba)) installation
 - The [SNAP and Sentinel toolboxes](https://step.esa.int/main/download/snap-download/) 

The Python 3.6 environment is defined in the [environment.yml](../environment.yml) file distributed with this package.


### Main setup steps:
 1. Create a working enviornment
  ```
  conda create -f environment.yml
  ```
 2. Install SNAP and follow the instructions to link it with the python environment you created above. 
    1. Open a SNAP command Line
    2. use the `snappy-conf` command with the following arguments
    ```
        snappy-conf Python Dir
    ```
    with the following arguments:
    - `Python`: Full path to Python executable to be used with SNAP. e.g.        <your_home>\mambaforge\envs\insar\python.exe
    - `Dir`:    Directory where the 'snappy' module should be installed e.g.   <your_home>\mambaforge\envs\insar\python\Lib.
    









In [1]:
import sys
import os
import shutil
import glob

sys.path.append("../") # go to parent dir

## Get data

### SAR

In [ ]:
from data import download_2015NepalEarthquake as dl

downloader = dl.bulk_downloader()
downloader.download_files()

for file in glob.glob("*.zip"):
    new_file = os.path.join("..", "data", file)
    shutil.copy(file, new_file)
    if os.path.isfile(new_file):
        os.remove(file)

### Weather

## Preprocess data

### SAR


#### `snappy` pipelines

In [5]:
from sar import InSAR_Pipeline as isp
import os

file_prefix = "../infiles"
output_prefix = "../outfiles"
scene = "kumamoto_2016"
IW =  "IW1"

firstBurstIndex = 1
lastBurstIndex = 9
ML_nRgLooks = 6

output_dir = scene + "_" + IW + "_" + "wsl"
output_path = os.path.join(output_prefix, output_dir)

os.makedirs(output_path, exist_ok=True)

# Nepal 2015 nepal_2015
if scene == "nepal_2015":
    filename_1 = os.path.join(file_prefix,'S1A_IW_SLC__1SSV_20150417T001852_20150417T001922_005516_0070C1_460B.zip')
    filename_2 = os.path.join(file_prefix,'S1A_IW_SLC__1SDV_20150429T001907_20150429T001935_005691_0074DC_7332.zip')

# Rockridge 2019 rockridge_2019
elif scene == "rockridge_2019":
    filename_1 = os.path.join(file_prefix,'S1B_IW_SLC__1SDV_20190626T020647_20190626T020714_016861_01FB9D_6003.zip')
    filename_2 = os.path.join(file_prefix,'S1B_IW_SLC__1SDV_20190708T020648_20190708T020715_017036_0200CC_D043.zip')

# Kumamoto Earthquake 2016 kumamoto_2016
elif scene == "kumamoto_2016":
    filename_1 = os.path.join(file_prefix,'S1A_IW_SLC__1SSV_20160408T091355_20160408T091430_010728_01001F_83EB.zip')
    filename_2 = os.path.join(file_prefix,'S1A_IW_SLC__1SSV_20160420T091355_20160420T091423_010903_010569_F9CE.zip')

else:
    raise RuntimeError("Specify a valid scene")


# Define input/output files
out_filename_i =os.path.join(output_path, 'InSAR_pipeline_I')

in_filename_ii =os.path.join(output_path, 'InSAR_pipeline_I.dim')
out_filename_ii =os.path.join(output_path, 'InSAR_pipeline_II')

in_filename_iii = os.path.join(output_path, 'InSAR_pipeline_II.dim')
out_filename_iii =os.path.join(output_path, 'InSAR_pipeline_III')

in_filename_iv = os.path.join(output_path, 'InSAR_pipeline_II.dim')
out_filename_iv =os.path.join(output_path, 'InSAR_pipeline_IV_snaphu_test')


# Call the processing pipeline step by step
isp.InSAR_pipeline_I(filename_1, filename_2, IW, firstBurstIndex, lastBurstIndex, out_filename_i)

isp.InSAR_pipeline_II(in_filename_ii, ML_nRgLooks, out_filename_ii)

isp.InSAR_pipeline_III(in_filename_iii, out_filename_iii)

isp.InSAR_pipeline_IV(in_filename_iv, out_filename_iv)

Reading...
Reading...
Apply TOPSAR Split...
Apply TOPSAR Split...
Applying orbit file ...


#### Phase unwrapping

Phase unwrapping is conducted with `snaphu` outside of the SNAP pipelines. It is ran from the console, using
``` bash
    $ snaphu n-f snapu.conf Phase_VV_<file>.img <num>
```
the the full namme of `<file>` and the number `<num>` is found at line 7 of the `snaphu.conf` file created by the last processing step.

**NOTE** As of submission time this step must be done manually, but it be easily scriptable in python.